### References:
* https://flask-sqlalchemy.readthedocs.io/en/stable/
* https://docs.sqlalchemy.org/en/20/orm/inheritance.html#concrete-table-inheritance
* https://docs.sqlalchemy.org/en/20/_modules/examples/performance/bulk_inserts.html
* https://docs.sqlalchemy.org/en/20/orm/large_collections.html#bulk-insert-of-new-items
* https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html

## Refactoring notes

- Dans l'objet PremiumFile, il manque la relation à une analyse (mais la table d'association existe bien) :

```python
class PremiumFile(CommonMixin, Base):
    """Represents a historical premium file."""
    id: Mapped[int] = mapped_column(primary_key=True)
    client_id: Mapped[int] = mapped_column(ForeignKey("client.id"))
    client: Mapped["Client"] = relationship(back_populates="premiumfiles")

    # TODO: To be added
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_premiumfile_table, back_populates="premiumfiles"
    )
```

- Tester les suppressions d'objets
- Associer threshold au model file et aux distributions de fréquence et de sévérité
- The relationship between a frequency severity model and the input premium file is missing :

```python
class FrequencySeverityModel(ModelFile):
    id: Mapped[int] = mapped_column(ForeignKey("modelfile.id"), primary_key=True)
    threshold: Mapped[int] = mapped_column(nullable=False)
    lossfile_id: Mapped[int] = mapped_column(ForeignKey("histolossfile.id"))
    lossfile: Mapped["HistoLossFile"] = relationship()
    premiumfile_id: Mapped[int] = mapped_column(ForeignKey("premiumfile.id"))  # TODO: To be added
    premiumfile: Mapped["PremiumFile"] = relationship()  # TODO: To be added
```

- In FrequencySeverityModel, the columns frequencymodel_id and severity_models_id were missing => it was possible to delete frequencymodels or severitymodels linked to a frequencyseveritymodel

```python
class FrequencySeverityModel(ModelFile):
    frequencymodel_id: Mapped[int] = mapped_column(ForeignKey("frequencymodel.id"))  # TODO: To be added
    frequencymodel: Mapped["FrequencyModel"] = relationship(
        # back_populates="frequencyseveritymodel", cascade="all, delete-orphan"  # TODO: To be deleted
        back_populates="frequencyseveritymodel"  # TODO: To be corrected
    )
    severitymodel_id: Mapped[int] = mapped_column(ForeignKey("severitymodel.id"))  # TODO: To be added
    severitymodel: Mapped["SeverityModel"] = relationship(
        back_populates="frequencyseveritymodel"  # TODO: To be corrected
    )
```

```python
class FrequencyModel(CommonMixin, Base):
    # frequencyseveritymodel_id: Mapped[int] = mapped_column(
    #     ForeignKey("frequencyseveritymodel.id"), nullable=False
    # )  # TODO: To be deleted
    frequencyseveritymodel: Mapped["FrequencySeverityModel"] = relationship(
        back_populates="frequencymodel"
    )
```

```python
class SeverityModel(CommonMixin, Base):
    # frequencyseveritymodel_id: Mapped[int] = mapped_column(
    #     ForeignKey("frequencyseveritymodel.id"), nullable=False
    # )  # TODO: To be deleted
    frequencyseveritymodel: Mapped["FrequencySeverityModel"] = relationship(
        back_populates="severitymodel"
    )
```

- The back-relationship were missing for handling properly session.delete(histolossfile) :
- The back-relationship were missing for handling properly session.delete(histolossfile) :

```python
class Analysis(CommonMixin, Base):
    id: Mapped[int] = mapped_column(primary_key=True)
    client_id: Mapped[int] = mapped_column(ForeignKey("client.id"), nullable=False)
    client: Mapped["Client"] = relationship(back_populates="analyses")
    
    histolossfiles: Mapped[List["HistoLossFile"]] = relationship(
        secondary=lambda: analysis_histolossfile_table, back_populates="analyses"
    )
    modelfiles: Mapped[List["ModelFile"]] = relationship(
        secondary=lambda: analysis_modelfile_table, back_populates="analyses"
    )
    
class HistoLossFile(CommonMixin, Base):
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_histolossfile_table, back_populates="histolossfiles"
    )

class ModelFile(CommonMixin, Base):
    id: Mapped[int] = mapped_column(primary_key=True)
    model_type: Mapped[str] = mapped_column(String(50), nullable=False)
    years_simulated: Mapped[int] = mapped_column(nullable=False)
    
    client_id: Mapped[int] = mapped_column(ForeignKey("client.id"), nullable=False)
    client: Mapped["Client"] = relationship(back_populates="modelfiles")
    
    yearlosses: Mapped[List["ModelYearLoss"]] = relationship(
        back_populates="modelfile",
        cascade="all, delete-orphan",
    )
    
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_modelfile_table, back_populates="modelfiles"
    )
```

- Cascade delete, all for client-analysis:

```python
class Client(CommonMixin, Base): """Represents a client entity."""

  id: Mapped[int] = mapped_column(primary_key=True)
  name: Mapped[str] = mapped_column(String(50), nullable=False)

  analyses: Mapped[List["Analysis"]] = relationship(
      back_populates="client", cascade="all, delete-orphan"
  )
```

- The Pydantic classes FrequencyInput and SeverityInput need to be reviewed and refactored
- The attribute treshold is missing in the FrequencyModel class
- Start frequency and severity models parameters with index 0
- The threshold of frequency and severity models is that of the related frequency_severity_model => Remove attribute threshold from severity model
- Use ModelType in polymorphic identity

- Confusion entre input et output (cf. https://gitlab.com/ccr-re-df/products/app/backends/tarification-nonvie-backend/-/blob/dev/app/api/routes/model.py) :

```python
async def generate_stochastic_year_loss_table_and_metadata(
    frequence_model_input: FrequencyModelOutput,
    severity_model_input: SeverityModelOutput,
    frequence_severity_model_input: FrequencySeverityModelInputExtend,
    threshold_input: float,
    analysis_id: int,
    user_id: int,
    lossfile_id: int,
    injected_model_service: ModelServiceDep,
) -> StochasticModelRouteResponse:
    """
    Generate stochastic year loss table along with metadata.

    Args:
        frequence_model_input (FrequencyModelOutput): Output from the frequency model.
        severity_model_input (SeverityModelOutput): Output from the severity model.

```

- ADD PREMIUM FILE ID
- FIX THE PROBLEM WITH DELETING A FREQUENCY SOLO, A SEVERITY SOLO
- THEN CREATE JIRA SPECIFIC ISSUES
- THEN WORKSHOP WITH ANTOINE B TO REVIEW CHANGES LIKE THOSE IN PYDANTIC FOR FREQUENCYINPUT SEVERITYINPUT ETC

## Test Engine

In [1]:
import time

from engine.model.frequency_severity import (
    DistributionInput,
    DistributionType,
    get_modelyearloss_frequency_severity,
)

In [2]:
threshold = 1000

frequency_input = DistributionInput(
    dist=DistributionType.POISSON,
    threshold=threshold,
    params=[3],
)

severity_input = DistributionInput(
    dist=DistributionType.PARETO,
    threshold=threshold,
    params=[2],
)

cat_share = 0.5
simulated_years = 100_000
modelfile_id = 1

start = time.perf_counter()

modelyearloss = get_modelyearloss_frequency_severity(
    frequency_input,
    severity_input,
    cat_share,
    simulated_years,
    modelfile_id,
)

print(f"Duration = {time.perf_counter() - start}")
print(f"Average Loss = {modelyearloss["loss"].mean()}")
print(f"Frequency = {len(modelyearloss["loss"]) / simulated_years}")
print(modelyearloss)

Duration = 0.1773916999809444
Average Loss = 1986.6839583027847
Frequency = 3.00068
shape: (300_068, 11)
┌───────┬─────┬──────┬───────────┬───┬────────────┬────────┬──────────────────┬──────────────┐
│ year  ┆ day ┆ loss ┆ loss_type ┆ … ┆ model_hash ┆ model  ┆ line_of_business ┆ modelfile_id │
│ ---   ┆ --- ┆ ---  ┆ ---       ┆   ┆ ---        ┆ ---    ┆ ---              ┆ ---          │
│ i64   ┆ i32 ┆ i64  ┆ str       ┆   ┆ object     ┆ object ┆ object           ┆ i64          │
╞═══════╪═════╪══════╪═══════════╪═══╪════════════╪════════╪══════════════════╪══════════════╡
│ 0     ┆ 152 ┆ 1534 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 261 ┆ 1232 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 91  ┆ 1699 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 91  ┆ 1923 ┆ non_cat   ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 1     ┆ 61  ┆ 1000 ┆ cat       ┆ … ┆ n

## Test Backend

In [2]:
import time

from db.crud import (
    create_analysis,
    create_client,
    create_frequency_severity_model,
    create_historical_loss_file,
    create_premium_file,
    delete_db_record,
)
from db.models import (
    Analysis,
    Client,
    FrequencyModel,
    FrequencySeverityModel,
    HistoLossFile,
    ModelFile,
    ModelYearLoss,
    PremiumFile,
    SeverityModel,
    session,
)

In [3]:
# Create a client, an analysis, a premium file and a historical loss file
create_client(session, client_name="AXA")
create_analysis(session, client_id=1)
create_premium_file(session, analysis_id=1)
create_historical_loss_file(session, analysis_id=1)

Client 'AXA' added successfully.
Analysis added successfully.
Premium file added successfully.
Historical loss file added successfully.


In [5]:
# Create a frequency-severity model
start = time.perf_counter()
create_frequency_severity_model(
    session,
    analysis_id=1,
    lossfile_id=1,
    premiumfile_id=1,
    threshold=1000,
    frequency_input=DistributionInput(
        dist=DistributionType.POISSON,
        threshold=1000,
        params=[3, 0, 0, 0, 0],
    ),
    severity_input=DistributionInput(
        dist=DistributionType.PARETO,
        threshold=1000,
        params=[2, 0, 0, 0, 0],
    ),
    cat_share=0.5,
    years_simulated=10_000,
)

duration = time.perf_counter() - start
print(f"Total Duration = {duration}")

Time to create model file: 0.05 seconds
Time to flush the session: 0.00 seconds
Time to generate year loss data: 0.05 seconds
Time to insert year loss records into database: 0.75 seconds
Time to commit transaction: 0.05 seconds
Frequency-Severity Model created successfully.
Total Duration = 0.9006514998618513


In [10]:
# DbModel = PremiumFile
# DbModel = HistoLossFile
# DbModel = FrequencyModel
DbModel = SeverityModel

delete_db_record(session, DbModel, 1)

An unexpected error occurred while deleting SeverityModel record: SeverityModel record with ID 1 not found.


ValueError: SeverityModel record with ID 1 not found.

- Problem with the deletion of frequency and severity models